In [11]:
import os
import requests
import base64
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

api_key="FmIgf7VyuYfjWQv2232Ipcqlu9f6fwy7HQZO0G0eyXGOnqvruEWkJQQJ99BDACfhMk5XJ3w3AAABACOGmclz"

IMAGE_PATH = "data/complex_report_screenshot.png"

encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that extract information from images."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_image}"
          }
        },
        {
          "type": "text",
          "text":"explain this image and give me launch date, fund size and risk class as key value pairs."
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

uri = "https://swedencentral-shared-yk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview"

# Send request
try:
    response = requests.post(uri, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'annotations': [], 'content': "This image is a fact sheet for the **Principal - MPF Conservative Fund (CAP)**, which provides details about the fund's performance, investment objectives, allocation, and holdings as of 31/12/2023. Key sections include Fund Details, Investment Objective, Fund Performance, Allocation, and Commentary.\n\nHere are the requested key-value pairs:\n\n- **Launch Date**: 1/12/2000  \n- **Fund Size (in Million)**: HKD 3,804.49  \n- **Risk Class**: 1  ", 'refusal': None, 'role': 'assistant'}}], 'created': 1749762918, 'id':

In [9]:
import json
result = json.loads(response.text)["choices"][0]["message"]["content"]

In [10]:
result

'This image is a detailed fund fact sheet for the "Principal - MPF Conservative Fund (CAP)," provided by Principal MPF. It contains various sections that inform an investor about the fund\'s characteristics, performance, investment allocations, and holdings. The document includes:\n\n- **Fund Details**: Basic information such as fund descriptor, management company, launch date, and size.\n- **Investment Objective**: Describes the purpose and investment goals of the fund.\n- **Balance of Investments**: Explains how the fund achieves its investment objectives.\n- **Fund Performance**: Displays the performance metrics over different time periods.\n- **Fund Commentary**: Offers a market commentary and the fund manager\'s insights.\n- **Fund Allocation by Asset Class**: Shows a pie chart distribution of investments across different asset classes.\n- **Top 10 Holdings**: Lists the top assets held within the fund.\n\nHere are the key details requested:\n\n- **Launch Date**: 1/12/2000\n- **Fun

# JSON FORMAT OUTPUT

In [1]:
import datetime
import json
import openai
import os
import sys

from openai import AzureOpenAI
from dotenv import load_dotenv
from pprint import pprint

In [2]:
# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [3]:
completion = client.chat.completions.create(
    model="gpt-4o", #"gpt-4turbo-0409",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Your response should be in JSON format.",
        },
        {
            "role": "user",
            "content": "List me 10 top movies from the 70's with the title, the director, the year \
        and the main actors",
        },
    ],
    response_format={"type": "json_object"},
)

In [4]:
print(completion)

ChatCompletion(id='chatcmpl-BhjR2I5Kdc0V0e0yP2BQ4UcD863d9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "top_70s_movies": [\n    {\n      "title": "The Godfather",\n      "director": "Francis Ford Coppola",\n      "year": 1972,\n      "main_actors": ["Marlon Brando", "Al Pacino", "James Caan"]\n    },\n    {\n      "title": "Star Wars: Episode IV - A New Hope",\n      "director": "George Lucas",\n      "year": 1977,\n      "main_actors": ["Mark Hamill", "Harrison Ford", "Carrie Fisher"]\n    },\n    {\n      "title": "One Flew Over the Cuckoo\'s Nest",\n      "director": "Milos Forman",\n      "year": 1975,\n      "main_actors": ["Jack Nicholson", "Louise Fletcher", "William Redfield"]\n    },\n    {\n      "title": "Jaws",\n      "director": "Steven Spielberg",\n      "year": 1975,\n      "main_actors": ["Roy Scheider", "Robert Shaw", "Richard Dreyfuss"]\n    },\n    {\n      "title": "Apocalypse Now",\n      "director": "F

In [5]:
completion.usage

CompletionUsage(completion_tokens=556, prompt_tokens=50, total_tokens=606, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [6]:
json_answer = json.loads(completion.choices[0].message.content)
pprint(json_answer)

{'top_70s_movies': [{'director': 'Francis Ford Coppola',
                     'main_actors': ['Marlon Brando',
                                     'Al Pacino',
                                     'James Caan'],
                     'title': 'The Godfather',
                     'year': 1972},
                    {'director': 'George Lucas',
                     'main_actors': ['Mark Hamill',
                                     'Harrison Ford',
                                     'Carrie Fisher'],
                     'title': 'Star Wars: Episode IV - A New Hope',
                     'year': 1977},
                    {'director': 'Milos Forman',
                     'main_actors': ['Jack Nicholson',
                                     'Louise Fletcher',
                                     'William Redfield'],
                     'title': "One Flew Over the Cuckoo's Nest",
                     'year': 1975},
                    {'director': 'Steven Spielberg',
                  

In [8]:
json_answer["top_70s_movies"][0]

{'title': 'The Godfather',
 'director': 'Francis Ford Coppola',
 'year': 1972,
 'main_actors': ['Marlon Brando', 'Al Pacino', 'James Caan']}

In [9]:
for i in range(len(json_answer["top_70s_movies"])):
    print(i+1, json_answer["top_70s_movies"][i]["title"], json_answer["top_70s_movies"][i]["year"])

1 The Godfather 1972
2 Star Wars: Episode IV - A New Hope 1977
3 One Flew Over the Cuckoo's Nest 1975
4 Jaws 1975
5 Apocalypse Now 1979
6 Taxi Driver 1976
7 A Clockwork Orange 1971
8 Rocky 1976
9 The Exorcist 1973
10 Chinatown 1974
